<a href="https://colab.research.google.com/github/msaligane/US_Japan_Semiconductor_Workshop/blob/main/Day%202%20-%201505%20-%20Pitfalls%20of%20Open-Source%20Chip%20Design%20Verification/open_source_BE_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pitfalls of Open-Source Chip Design Verification<br>オープンソース設計検証での注意点
US-Japan Semiconductor Workshop, May 15th, 2024<br>最先端集積回路設計に関する日米連携ワークショップ 2024年5月15日

D. Mitch Bailey　ShuhariSystem<br>ベイリー・デビッド　シュハリシステム

# Overview  概要
Open source EDA software and systems such as OpenROAD, yosys, magic, xschem, klayout, iverilog, hgspice, etc., have made it possible for anyone to design and layout a mixed-signal chip. While the final step of verifying that the design matches the layout is possible with open source tools, there are pitfalls that you should be aware of. This workshop will provide hands-on experience with back-end verification using actual designs that were taped out on the Google funded open-MPW shuttles.

OpenROAD, yosys, magic, xschem, klayout, iverilog, hgspice などのオープンソースソフトウェアやシステムのお陰で、誰でもがアナログやディジタルのチップを設計して、レイアウトすることができます。最終的のデザインとレイアウトの一致を確認するステップもオープンソースのツールでできますが要注意点があります。このワークショップでは実際にGoogleのOpen-MPWで出されたチップのデータを各自で検証できます。

# Google Open-MPW repos
The Google open mpw repos are located [here](https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130). There were 8 shuttles with 40 designs each, so there are 320 designs to choose from. The designs for each shuttle are summarized in a `manifest.csv` file of the `foundry` repo of each shuttle. For example, the list of designs for `mpw-003` are [here](https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130/mpw-003/foundry/+/refs/heads/main/manifest-mpw-003.csv).

GoogleのオープンMPWのrepoは[ここ](https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130)にあります。8回のシャトルがあって、各シャトルに40個のチップが乗っていたので合計320デザインから選べます。各シャトルに乗っているデザインがそれぞれの`foundry` repoの`manifest.csv`にリストされています。例として、`mpw-003`のデザインリストは[ここ](https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130/mpw-003/foundry/+/refs/heads/main/manifest-mpw-003.csv)にあります。

For previously published results see the following discussions:<br>
すでに発表された検証結果を以下で確認できます：
* [sky130/mpw-002](https://github.com/d-m-bailey/extra_be_checks/discussions/26)
* [sky130/mpw-003](https://github.com/d-m-bailey/extra_be_checks/discussions/27)
* [sky130/mpw-004](https://github.com/d-m-bailey/extra_be_checks/discussions/35)
* [sky130/mpw-006](https://github.com/d-m-bailey/extra_be_checks/discussions/39)


In [1]:
%%writefile /content/env
export PDK_ROOT=/content/pdks
export PDK=sky130B
export PDKPATH=$PDK_ROOT/$PDK
export PRECHECK_ROOT=/content/mpw_precheck
export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=mpw-005
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT

Writing /content/env


# User setup
Only needs to be executed once.

This step sets up docker, pdk, precheck and creates a new user named lvs. <br><b>NOTE:</b> You will be prompted for a password - we suggest using lvs. Other info can be blank.

In [2]:
%%shell
cat /content/env
source /content/env
pip install udocker
pip install volare
udocker --allow-root install
rm -rf mpw_precheck
git clone https://github.com/efabless/mpw_precheck.git $PRECHECK_ROOT
volare enable 6d4d11780c40b20ee63cc98e645307a9bf2b2ab8
# remove unused pdk
rm -rf $PDK_ROOT/sky130A
rm -rf extra_be_checks
git clone https://github.com/d-m-bailey/extra_be_checks.git -b 2406
adduser lvs
usermod -aG sudo lvs

export PDK_ROOT=/content/pdks
export PDK=sky130B
export PDKPATH=$PDK_ROOT/$PDK
export PRECHECK_ROOT=/content/mpw_precheck
export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=mpw-005
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.16
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools succ

Create a script to extract the user_project_wrapper hierarchy as gds from the caravel oasis.

Only needs to be run once.

In [3]:
%%writefile oas2gds.py
import pya

app = pya.Application.instance()
opt = pya.SaveLayoutOptions()
layout_view = pya.Layout()

input_layout = "caravel.oas"
# Setting the name of the output file and setting the substitution character
print("[INFO] Reading from " + input_layout)

# Reading the input file and writing it to the output file name
layout_view.read(input_layout)
for cell_it in layout_view.each_cell():
    if cell_it.name.endswith("project_wrapper"):
        myIndex = layout_view.cell(cell_it.name).cell_index()
        output = "work/gds/" + cell_it.name + ".gds.gz"
        opt.set_format_from_filename(output)
        opt.oasis_substitution_char=''
        break
opt.select_cell(myIndex)
print("[INFO] Writing " + output)
layout_view.write(output, opt)

app.exit(0)

Writing oas2gds.py


# Clone the design repo and create gds.

In [4]:
%%shell
cat /content/env
source /content/env
rm -rf data
git clone https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130/$MPW/$SLOT.git data
rm -rf data/.git data/tapeout/outputs/gds
if [[ -d oas ]]; then
  ln -s $PWD/data/oas/cara*oas data/caravel.oas
else
  ln -s $PWD/data/tapeout/outputs/oas/cara*_*oas data/caravel.oas
fi
mkdir -p data/work/gds
cd data
cp ../oas2gds.py .
chown -R lvs:lvs .
sudo -u lvs udocker run -i -t -v $PWD:$PWD \
  -u $(id -u lvs):$(id -g lvs) \
  efabless/mpw_precheck:latest bash -c "cd $PWD ; klayout -b -rm oas2gds.py"

export PDK_ROOT=/content/pdks
export PDK=sky130B
export PDKPATH=$PDK_ROOT/$PDK
export PRECHECK_ROOT=/content/mpw_precheck
export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=mpw-005
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT
Cloning into 'data'...
remote: Total 1546 (delta 932), reused 1546 (delta 932)
Receiving objects: 100% (1546/1546), 520.25 MiB | 27.37 MiB/s, done.
Resolving deltas: 100% (932/932), done.
Updating files: 100% (164/164), done.
Info: creating repo: /home/lvs/.udocker
Info: udocker command line interface 1.3.16
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful
Info: downloading layer sha256:9ffd175d09c2931ac1d0689af8492615f7ea1cf9daece4427cb06d3e7caf78df
Info: downloading layer sha256:1c995a21e29168ce44d4791f9dfe3b1e5ab4e0e79166fcd5ee8b563d6928dae1
Info: downloading layer sha256:4f4fb700ef54461cfa025

# List verilog and spice files

In [5]:
%%shell
source /content/env
cd data
find . -name '*.spice'
find . -name '*.v'

./spi/lvs/user_project_wrapper.spice
./spi/lvs/user_proj_example.spice
./mpw_precheck/outputs/user_project_wrapper.filtered.v
./verilog/rtl/uprj_netlists.v
./verilog/rtl/main.v
./verilog/rtl/user_project_wrapper.v
./verilog/rtl/L1_cache.v
./verilog/rtl/user_proj_example.v
./verilog/rtl/memory_trace.v
./verilog/gl/user_project_wrapper.v
./verilog/gl/user_proj_example.v
./verilog/dv/la_test/la_test_tb.v
./tapeout/outputs/verilog/rtl/caravel.v
./tapeout/outputs/verilog/gl/gpio_defaults_block_0403.v
./tapeout/outputs/verilog/gl/gpio_defaults_block_1803.v
./tapeout/outputs/verilog/gl/caravel.v
./tapeout/outputs/verilog/gl/user_id_programming.v


# Create lvs_config.json

Update the TOP_LAYOUT, LVS_SPICE_FILES, LVS_VERILOG_FILES, and LAYOUT_FILE for every design.

Update the other parameters as needed.

In [6]:
%%writefile lvs_config.json
{
        "STD_CELL_LIBRARY": "sky130_fd_sc_hd",
        "INCLUDE_CONFIGS": [
                "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
        ],
        "TOP_SOURCE": "$TOP",
        "TOP_LAYOUT": "SJ_user_project_wrapper",
        "EXTRACT_FLATGLOB": [
                ""
        ],
        "EXTRACT_ABSTRACT": [
                ""
        ],
        "LVS_FLATTEN": [
                ""
        ],
        "LVS_NOFLATTEN": [
                ""
        ],
        "LVS_IGNORE": [
                ""
        ],
        "LVS_SPICE_FILES": [
                ""
        ],
        "LVS_VERILOG_FILES": [
                "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
                "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
        ],
        "LAYOUT_FILE": "$UPRJ_ROOT/work/gds/SJ_user_project_wrapper.gds.gz"
}

Writing lvs_config.json


# Run BE checks

In [17]:
%%shell
cat /content/env
source /content/env
cd data
gdspath=$(ls work/gds/*project_wrapper.gds.gz)
gdsfile=${gdspath##*/}
layout=${gdsfile%%.gds.gz}
source=${layout//[A-Z0-9][A-Z0-9]_}
echo "Layout is $layout, source is $source"
cp $LVS_ROOT/tech/$PDK/cvc.power.$source work/cvc.power.$layout
cp ../lvs_config.json lvs_config.json
export INPUT_DIRECTORY=$PWD
sudo -u lvs udocker run -i -t -v $PRECHECK_ROOT:$PRECHECK_ROOT \
  -v $INPUT_DIRECTORY:$INPUT_DIRECTORY \
  -v $LVS_ROOT:$LVS_ROOT \
  -v $PDK_ROOT:$PDK_ROOT \
  -e INPUT_DIRECTORY=$INPUT_DIRECTORY \
  -e PDK_PATH=$PDK_ROOT/$PDK \
  -e PDK=$PDK \
  -e TOP=$TOP \
  -e PDK_ROOT=$PDK_ROOT \
  -e UPRJ_ROOT=$UPRJ_ROOT \
  -e PDKPATH=$PDKPATH \
  -e LVS_ROOT=$LVS_ROOT \
  -e WORK_ROOT=$INPUT_DIRECTORY/work \
  -u $(id -u lvs):$(id -g lvs) \
  efabless/mpw_precheck:latest bash -c "env; cd $INPUT_DIRECTORY ; $LVS_ROOT/run_be_checks lvs_config.json"
gunzip -f work/cvc.error.gz
head -1 work/cvc.oeb.report > work/cvc.oeb.sort.report
tail -n +2 work/cvc.oeb.report | sort -k1,1n >> work/cvc.oeb.sort.report

Streaming output truncated to the last 5000 lines.
  Class: sky130_fd_pr__nfet_01v8 instances:   8
  Class: sky130_fd_pr__pfet_01v8_hvt instances:  12
  Class: sky130_fd_pr__special_nfet_01v8 instances:   4
Circuit contains 16 nets.

Circuit 1 contains 24 devices, Circuit 2 contains 24 devices.
Circuit 1 contains 16 nets,    Circuit 2 contains 16 nets.


Contents of circuit 1:  Circuit: 'SJ_QD_sky130_fd_sc_hd__decap_3'
Circuit SJ_QD_sky130_fd_sc_hd__decap_3 contains 2 device instances.
  Class: sky130_fd_pr__nfet_01v8 instances:   1
  Class: sky130_fd_pr__pfet_01v8_hvt instances:   1
Circuit contains 2 nets.
Contents of circuit 2:  Circuit: 'SJ_QD_sky130_fd_sc_hd__decap_3'
Circuit SJ_QD_sky130_fd_sc_hd__decap_3 contains 2 device instances.
  Class: sky130_fd_pr__nfet_01v8 instances:   1
  Class: sky130_fd_pr__pfet_01v8_hvt instances:   1
Circuit contains 2 nets.

Circuit 1 contains 2 devices, Circuit 2 contains 2 devices.
Circuit 1 contains 2 nets,    Circuit 2 contains 2 nets.


Conte

# View verification logs

In [10]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')
files.view('/content/data/work/cvc.oeb.log')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# View soft verification report

In [11]:
from google.colab import files

files.view('/content/data/work/soft.report')

<IPython.core.display.Javascript object>

# View LVS report

In [20]:
from google.colab import files

files.view('/content/data/work/lvs.report')

<IPython.core.display.Javascript object>

# View CVC report

In [19]:
from google.colab import files

files.view('/content/data/work/cvc.error')

<IPython.core.display.Javascript object>

# View OEB report

In [18]:
from google.colab import files

files.view('/content/data/work/cvc.oeb.sort.report')

<IPython.core.display.Javascript object>